In [1]:
import numpy as np

# Generate a random 3x3 matrix
random_matrix = np.random.rand(3, 3)

# Calculate the 2-norm of the matrix
norm_2 = np.linalg.norm(random_matrix, ord=2)

print("Random Matrix:")
print(random_matrix)
print("2-Norm:", norm_2)

Random Matrix:
[[0.39417661 0.60159794 0.24099094]
 [0.20476561 0.69805133 0.95957353]
 [0.83990868 0.27324809 0.85237213]]
2-Norm: 1.7522760464325124


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("/content/gdrive/MyDrive/PPHAU/YOLO_Object_Detection/YOLOv8_fine_tuning/")
!ls

config.yaml  images_all  labels_all  weights  YOLOv8_Object_Detection.ipynb


In [ ]:
!nvidia-smi

Sun Jul 16 22:08:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install ultralytics

In [ ]:
from PIL import Image
import cv2
import json
import random
import shutil

from ultralytics import YOLO
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.136 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 24.2/166.8 GB disk)


In [ ]:
ROOT_DIR = '/content/gdrive/MyDrive/PPHAU/YOLO_Object_Detection/YOLOv8_fine_tuning/'

In [ ]:
# Set the split percentages for train, val, and test
train_split = 0.80
val_split = 0.20
test_split = 0

# Set the paths to the relevant directories
yolov8_dir = os.getcwd()
all_images_dir = os.path.join(yolov8_dir, "images_all")
all_labels_dir = os.path.join(yolov8_dir, "labels_all")
data_dir = os.path.join(yolov8_dir, "data")

# Create the data directory
os.makedirs(data_dir, exist_ok=True)

# Create train, val, and test directories inside the data directory
for split in ["train", "val", "test"]:
    split_dir = os.path.join(data_dir, split)
    images_dir = os.path.join(split_dir, "images")
    labels_dir = os.path.join(split_dir, "labels")
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

# Get a list of all image files in the all_images directory
image_files = [f for f in os.listdir(all_images_dir) if f.endswith((".jpg"))]

# Shuffle the image files randomly
random.shuffle(image_files)
num_images = len(image_files)
num_train = int(num_images * train_split)
num_val = int(num_images * val_split)
num_test = num_images - num_train - num_val

# Copy the images and labels to the respective folders
for i, file in enumerate(image_files):
    image_file = os.path.join(all_images_dir, file)
    label_file = os.path.join(all_labels_dir, os.path.splitext(file)[0] + ".txt")

    # Determine the split
    if i < num_train:
        split = "train"
    elif i < num_train + num_val:
        split = "val"
    else:
        split = "test"

    split_images_dir = os.path.join(data_dir, split, "images")
    split_labels_dir = os.path.join(data_dir, split, "labels")

    # Copy the image and label files to the respective folders
    shutil.copy(image_file, os.path.join(split_images_dir, file))
    shutil.copy(label_file, os.path.join(split_labels_dir, os.path.splitext(file)[0] + ".txt"))

print("Images and labels split successfully!")

# Count the number of images and labels in each split
for split in ["train", "val", "test"]:
    split_images_dir = os.path.join(data_dir, split, "images")
    split_labels_dir = os.path.join(data_dir, split, "labels")

    num_images_split = len([f for f in os.listdir(split_images_dir) if f.endswith((".jpg"))])
    num_labels_split = len([f for f in os.listdir(split_labels_dir) if f.endswith(".txt")])

    print(f"Split: {split}")
    print(f"Number of images: {num_images_split}")
    print(f"Number of labels: {num_labels_split}")
    print()

Images and labels split successfully!
Split: train
Number of images: 160
Number of labels: 160

Split: val
Number of images: 40
Number of labels: 40

Split: test
Number of images: 1
Number of labels: 1



In [ ]:
for split in ["train", "val", "test"]:
    split_images_dir = os.path.join(data_dir, split, "images")
    split_labels_dir = os.path.join(data_dir, split, "labels")

    num_images_split = len([f for f in os.listdir(split_images_dir) if f.endswith((".jpg"))])
    num_labels_split = len([f for f in os.listdir(split_labels_dir) if f.endswith(".txt")])

    print(f"Split: {split}")
    print(f"Number of images: {num_images_split}")
    print(f"Number of labels: {num_labels_split}")
    print()

Split: train
Number of images: 160
Number of labels: 160

Split: val
Number of images: 40
Number of labels: 40

Split: test
Number of images: 1
Number of labels: 1



In [ ]:
%cd /content/gdrive/MyDrive/PPHAU/YOLO_Object_Detection/YOLOv8_fine_tuning/

/content


In [ ]:
model = YOLO('weights/yoloxcustom.pt')
train_results = model.train(data= '/content/gdrive/MyDrive/PPHAU/YOLO_Object_Detection/YOLOv8_fine_tuning/config.yaml', epochs=500, project="furniture_assembly")

Ultralytics YOLOv8.0.136 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=weights/yoloxcustom.pt, data=/content/gdrive/MyDrive/PPHAU/YOLO_Object_Detection/YOLOv8_fine_tuning/config.yaml, epochs=500, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=furniture_assembly, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

In [ ]:
model = YOLO('/content/gdrive/MyDrive/PPHAU/YOLO_Object_Detection/YOLOv8/furniture_assembly/train5/weights/best.pt')
val_results = model.val()

Ultralytics YOLOv8.0.136 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 268 layers, 68129346 parameters, 0 gradients
val: Scanning /content/gdrive/MyDrive/PPHAU/YOLO_Object_Detection/YOLOv8/data/val/labels.cache... 88 images, 0 backgrounds, 0 corrupt: 100%|██████████| 88/88 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [05:49<00:00, 58.28s/it]
                   all         88        579      0.899      0.897      0.918      0.832
                   leg         88        223      0.899      0.857        0.9      0.792
            tableplate         88         88      0.974      0.989      0.986      0.928
             left_hand         88         84      0.825      0.842      0.873        0.8
            right_hand         88         70      0.819      0.843      0.849      0.764
           screwdriver         88         55      0.959      0.927       0.95      0.816
  

In [ ]:
import os
from glob import glob
from PIL import Image

# Directory path
directory = "/content/gdrive/MyDrive/PPHAU/YOLO_Object_Detection/YOLOv8_fine_tuning/data/test/images/"

# Get all image paths in the directory
image_paths = glob(os.path.join(directory, "*.jpg"))

# Extract file names from image paths
file_names = [os.path.basename(path) for path in image_paths]

# Open the images
images = [Image.open(path) for path in image_paths]

model = YOLO('/content/gdrive/MyDrive/PPHAU/YOLO_Object_Detection/YOLOv8_fine_tuning/furniture_assembly/train2/weights/best.pt')

# Perform object detection on the images
test_results = model(images, conf=0.7, save= True, project="furniture_assembly")


0: 384x640 2 legs, 1 tableplate, 1 left_hand, 1 right_hand, 1 screwdriver, 1: 384x640 3 legs, 1 tableplate, 1 left_hand, 1 right_hand, 1 screwdriver, 1 drill, 2: 384x640 2 legs, 1 tableplate, 1 screwdriver, 1 drill, 3: 384x640 2 legs, 2 tableplates, 1 left_hand, 1 right_hand, 1 screwdriver, 1 drill, 4: 384x640 2 legs, 1 tableplate, 2 left_hands, 2 screwdrivers, 1 drill, 5: 384x640 2 legs, 1 tableplate, 1 left_hand, 1 right_hand, 1 drill, 6: 384x640 2 legs, 1 tableplate, 1 left_hand, 1 right_hand, 1 screwdriver, 1 drill, 7: 384x640 2 legs, 2 tableplates, 1 left_hand, 1 right_hand, 1 screwdriver, 1 drill, 8: 384x640 2 legs, 1 tableplate, 1 left_hand, 1 right_hand, 1 screwdriver, 1 drill, 9: 384x640 2 legs, 2 tableplates, 1 left_hand, 1 right_hand, 1 screwdriver, 1 drill, 10: 384x640 2 legs, 2 tableplates, 1 left_hand, 1 right_hand, 1 screwdriver, 1 drill, 11: 384x640 2 legs, 1 tableplate, 1 left_hand, 1 right_hand, 1 drill, 12: 384x640 2 legs, 1 tableplate, 1 left_hand, 1 right_hand, 1 

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob('furniture_assembly/predict4/*.jpg'):
      display(Image(filename=image_path, width=600))
      print("\n")

Output hidden; open in https://colab.research.google.com to view.